In [1]:
import requests
import json
from urllib import urlencode

In [2]:
requests.__version__

'2.9.1'

In [3]:
import pandas as pd

In [6]:
with open('../Data/players.json') as data_file:
    playid_data = json.load(data_file)

In [7]:
playid_df = pd.DataFrame(playid_data)

In [8]:
playid_df.head()

,firstName,lastName,playerId,teamId
0,Quincy,Acy,203112,1610612758
1,Jordan,Adams,203919,1610612763
2,Steven,Adams,203500,1610612760
3,Arron,Afflalo,201167,1610612752
4,Alexis,Ajinca,201582,1610612740


In [10]:
playid_df.shape

(468, 4)

There are only 468 players in the league

In [27]:
ids_ls = playid_df['playerId']

In [17]:
ids_ls[1:10]

1     203919
2     203500
3     201167
4     201582
5     202332
6     200746
7    1626146
8     202730
9       2754
Name: playerId, dtype: int64

# Player profile

In [24]:
class PlayerBio:
    def __init__(self, playerid, leagueid = '00'):
        self._url = "http://stats.nba.com/stats/commonplayerinfo?"
        self._api_param = (('PlayerID',playerid),('LeagueID',leagueid)) 
        u_a = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:43.0) Gecko/20100101 Firefox/43.0'}
        self._x = requests.get(self._url, params=self._api_param,headers={"USER-AGENT":u_a})
        self._x = self._x.json()
    def bio(self): # to collect only one player
        return pd.DataFrame(self._x['resultSets'][0]['rowSet'],columns=self._x['resultSets'][0]['headers'])

In [25]:
#test
tt = PlayerBio(str(playid_df['playerId'][0]))

In [26]:
tt.bio()

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,...,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,GAMES_PLAYED_FLAG
0,203112,Quincy,Acy,Quincy Acy,"Acy, Quincy",Q. Acy,1990-10-06T00:00:00,Baylor,USA,Baylor/USA,...,1610612758,Kings,SAC,kings,Sacramento,quincy_acy,2012,2015,Y,Y


In [28]:
def collectAllPlayerBios(ids_ls):
    leagueid = '00'
    url = "http://stats.nba.com/stats/commonplayerinfo?"
    u_a = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:43.0) Gecko/20100101 Firefox/43.0'} #header is necessary
    allplayerBios_df_ls = []
    
    for playerid in ids_ls:
        api_param = (('PlayerID',str(playerid)),('LeagueID',leagueid))
        response = requests.get(url, params=api_param,headers={"USER-AGENT":u_a})
        response_json = response.json()
        response_df = pd.DataFrame(response_json['resultSets'][0]['rowSet'],columns=response_json['resultSets'][0]['headers'])
        allplayerBios_df_ls.append(response_df)
        
    allplayerBios_df = pd.concat(allplayerBios_df_ls,axis=0)
    return(allplayerBios_df)

In [29]:
test_allplayerBios = collectAllPlayerBios(ids_ls[1:5])

In [30]:
test_allplayerBios

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,...,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,GAMES_PLAYED_FLAG
0,203919,Jordan,Adams,Jordan Adams,"Adams, Jordan",J. Adams,1994-07-08T00:00:00,UCLA,USA,UCLA/USA,...,1610612763,Grizzlies,MEM,grizzlies,Memphis,jordan_adams,2014,2015,Y,Y
0,203500,Steven,Adams,Steven Adams,"Adams, Steven",S. Adams,1993-07-20T00:00:00,Pittsburgh,New Zealand,Pittsburgh/New Zealand,...,1610612760,Thunder,OKC,thunder,Oklahoma City,steven_adams,2013,2015,N,Y
0,201167,Arron,Afflalo,Arron Afflalo,"Afflalo, Arron",A. Afflalo,1985-10-15T00:00:00,UCLA,USA,UCLA/USA,...,1610612752,Knicks,NYK,knicks,New York,arron_afflalo,2007,2015,N,Y
0,201582,Alexis,Ajinca,Alexis Ajinca,"Ajinca, Alexis",A. Ajinca,1988-05-06T00:00:00,"Saint Etienne, France",France,"Saint Etienne, France",...,1610612740,Pelicans,NOP,pelicans,New Orleans,alexis_ajinca,2008,2015,Y,Y


In [31]:
allPlayerBios = collectAllPlayerBios(ids_ls)

In [32]:
allPlayerBios.head(20)

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,...,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,GAMES_PLAYED_FLAG
0,203112,Quincy,Acy,Quincy Acy,"Acy, Quincy",Q. Acy,1990-10-06T00:00:00,Baylor,USA,Baylor/USA,...,1610612758,Kings,SAC,kings,Sacramento,quincy_acy,2012,2015,Y,Y
0,203919,Jordan,Adams,Jordan Adams,"Adams, Jordan",J. Adams,1994-07-08T00:00:00,UCLA,USA,UCLA/USA,...,1610612763,Grizzlies,MEM,grizzlies,Memphis,jordan_adams,2014,2015,Y,Y
0,203500,Steven,Adams,Steven Adams,"Adams, Steven",S. Adams,1993-07-20T00:00:00,Pittsburgh,New Zealand,Pittsburgh/New Zealand,...,1610612760,Thunder,OKC,thunder,Oklahoma City,steven_adams,2013,2015,N,Y
0,201167,Arron,Afflalo,Arron Afflalo,"Afflalo, Arron",A. Afflalo,1985-10-15T00:00:00,UCLA,USA,UCLA/USA,...,1610612752,Knicks,NYK,knicks,New York,arron_afflalo,2007,2015,N,Y
0,201582,Alexis,Ajinca,Alexis Ajinca,"Ajinca, Alexis",A. Ajinca,1988-05-06T00:00:00,"Saint Etienne, France",France,"Saint Etienne, France",...,1610612740,Pelicans,NOP,pelicans,New Orleans,alexis_ajinca,2008,2015,Y,Y
0,202332,Cole,Aldrich,Cole Aldrich,"Aldrich, Cole",C. Aldrich,1988-10-31T00:00:00,Kansas,USA,Kansas/USA,...,1610612746,Clippers,LAC,clippers,Los Angeles,cole_aldrich,2010,2015,Y,Y
0,200746,LaMarcus,Aldridge,LaMarcus Aldridge,"Aldridge, LaMarcus",L. Aldridge,1985-07-19T00:00:00,Texas,USA,Texas/USA,...,1610612759,Spurs,SAS,spurs,San Antonio,lamarcus_aldridge,2006,2015,N,Y
0,1626146,Cliff,Alexander,Cliff Alexander,"Alexander, Cliff",C. Alexander,1995-11-16T00:00:00,Kansas,USA,Kansas/USA,...,1610612757,Trail Blazers,POR,blazers,Portland,cliff_alexander,2015,2015,N,Y
0,202730,Lavoy,Allen,Lavoy Allen,"Allen, Lavoy",L. Allen,1989-02-04T00:00:00,Temple,USA,Temple/USA,...,1610612754,Pacers,IND,pacers,Indiana,lavoy_allen,2011,2015,N,Y
0,2754,Tony,Allen,Tony Allen,"Allen, Tony",T. Allen,1982-01-11T00:00:00,Oklahoma State,USA,Oklahoma State/USA,...,1610612763,Grizzlies,MEM,grizzlies,Memphis,tony_allen,2004,2015,N,Y


In [33]:
allPlayerBios.columns

Index([u'PERSON_ID', u'FIRST_NAME', u'LAST_NAME', u'DISPLAY_FIRST_LAST',
       u'DISPLAY_LAST_COMMA_FIRST', u'DISPLAY_FI_LAST', u'BIRTHDATE',
       u'SCHOOL', u'COUNTRY', u'LAST_AFFILIATION', u'HEIGHT', u'WEIGHT',
       u'SEASON_EXP', u'JERSEY', u'POSITION', u'ROSTERSTATUS', u'TEAM_ID',
       u'TEAM_NAME', u'TEAM_ABBREVIATION', u'TEAM_CODE', u'TEAM_CITY',
       u'PLAYERCODE', u'FROM_YEAR', u'TO_YEAR', u'DLEAGUE_FLAG',
       u'GAMES_PLAYED_FLAG'],
      dtype='object')

In [34]:
import pickle

In [35]:
with open('../Data/allPlayerBios.pickle', 'wb') as handle:
  pickle.dump(allPlayerBios, handle)

# Player Game Log

In [21]:
class GameLog:
    def __init__(self, playerid, season='2015-16',seasontype='Regular Season', leagueid='00'):
        self._url = "http://stats.nba.com/stats/playergamelog?"
        u_a = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:43.0) Gecko/20100101 Firefox/43.0'}
        self._api_param = (('LeagueID', leagueid),('PlayerID',playerid),('Season',season),('SeasonType',seasontype))
        self._x = requests.get(self._url, params=self._api_param,headers={"USER-AGENT":u_a})
        #print(self._x.raise_for_status)
        self._x = self._x.json()
    def log(self):
        return pd.DataFrame(self._x['resultSets'][0]['rowSet'],columns=self._x['resultSets'][0]['headers'])

In [22]:
tt_log = GameLog('203076')

In [23]:
tt_log.log()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22015,203076,0021500857,"FEB 25, 2016",NOP vs. OKC,W,38,11,26,0.423,...,5,6,2,1,4,2,2,30,1,1
1,22015,203076,0021500841,"FEB 23, 2016",NOP @ WAS,L,37,3,9,0.333,...,17,20,3,2,1,3,1,9,-20,1
2,22015,203076,0021500823,"FEB 21, 2016",NOP @ DET,W,43,24,34,0.706,...,14,20,4,0,1,2,1,59,17,1
3,22015,203076,0021500811,"FEB 19, 2016",NOP vs. PHI,W,38,9,22,0.409,...,5,7,1,2,2,2,3,34,-2,1
4,22015,203076,0021500802,"FEB 11, 2016",NOP @ OKC,L,31,8,14,0.571,...,3,3,2,2,0,1,3,23,-13,1
5,22015,203076,0021500798,"FEB 10, 2016",NOP vs. UTA,W,34,7,15,0.467,...,2,4,5,0,1,0,5,19,-10,1
6,22015,203076,0021500782,"FEB 08, 2016",NOP @ MIN,W,31,12,20,0.600,...,8,8,1,2,0,0,2,27,15,1
7,22015,203076,0021500763,"FEB 06, 2016",NOP @ CLE,L,40,11,20,0.550,...,7,11,0,0,2,2,3,24,-9,1
8,22015,203076,0021500747,"FEB 04, 2016",NOP vs. LAL,L,39,16,25,0.640,...,9,11,0,1,1,1,3,39,4,1
9,22015,203076,0021500742,"FEB 03, 2016",NOP @ SAS,L,37,11,22,0.500,...,7,10,4,4,4,4,1,28,-12,1


In [38]:
def collectAllPlayerGameLogs(ids_ls):
    leagueid = '00'
    url = "http://stats.nba.com/stats/playergamelog?"
    u_a = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:43.0) Gecko/20100101 Firefox/43.0'} #header is necessary
    allplayerGameLogs_df_ls = []
    season='2015-16'
    seasontype='Regular Season'
    
    for playerid in ids_ls:
        api_param = (('LeagueID', leagueid),('PlayerID',playerid),('Season',season),('SeasonType',seasontype))
        response = requests.get(url, params=api_param,headers={"USER-AGENT":u_a})
        response_json = response.json()
        response_df = pd.DataFrame(response_json['resultSets'][0]['rowSet'],columns=response_json['resultSets'][0]['headers'])
        allplayerGameLogs_df_ls.append(response_df)
        
    allplayerGameLogs_df = pd.concat(allplayerGameLogs_df_ls,axis=0)
    return(allplayerGameLogs_df)

In [42]:
allplayerGameLogs = collectAllPlayerGameLogs(ids_ls)

In [43]:
allplayerGameLogs.shape

(18964, 27)

In [45]:
len(allplayerGameLogs['Player_ID'].unique())

459

Double check the number of players. Still reasonable.

In [49]:
with open('../Data/allplayerGameLogs.pickle', 'wb') as handle:
  pickle.dump(allplayerGameLogs, handle)

In [50]:
allplayerGameLogs.columns

Index([u'SEASON_ID', u'Player_ID', u'Game_ID', u'GAME_DATE', u'MATCHUP', u'WL',
       u'MIN', u'FGM', u'FGA', u'FG_PCT', u'FG3M', u'FG3A', u'FG3_PCT', u'FTM',
       u'FTA', u'FT_PCT', u'OREB', u'DREB', u'REB', u'AST', u'STL', u'BLK',
       u'TOV', u'PF', u'PTS', u'PLUS_MINUS', u'VIDEO_AVAILABLE'],
      dtype='object')

In [51]:
allplayerGameLogs.dtypes

SEASON_ID           object
Player_ID          float64
Game_ID             object
GAME_DATE           object
MATCHUP             object
WL                  object
MIN                float64
FGM                float64
FGA                float64
FG_PCT             float64
FG3M               float64
FG3A               float64
FG3_PCT            float64
FTM                float64
FTA                float64
FT_PCT             float64
OREB               float64
DREB               float64
REB                float64
AST                float64
STL                float64
BLK                float64
TOV                float64
PF                 float64
PTS                float64
PLUS_MINUS         float64
VIDEO_AVAILABLE    float64
dtype: object